In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import matplotlib.pyplot as plt
import os

from keras.layers import *
from keras.models import Model
import keras

In [2]:
dataset = pd.read_pickle("./dataset.pkl")
dataset

,path,ageRange
13677,./UTKFace/1_1_0_20170109191253730.jpg.chip.jpg,0
1447,./UTKFace/1_1_2_20161219162225422.jpg.chip.jpg,0
10405,./UTKFace/1_0_2_20161219200203132.jpg.chip.jpg,0
11114,./UTKFace/1_0_1_20170110213647161.jpg.chip.jpg,0
12083,./UTKFace/1_0_0_20161219154724341.jpg.chip.jpg,0
...,...,...
16104,./UTKFace/90_1_0_20170120225317689.jpg.chip.jpg,90
8015,./UTKFace/92_1_0_20170110183501116.jpg.chip.jpg,90
23619,./UTKFace/90_1_0_20170120225510266.jpg.chip.jpg,90
13965,./UTKFace/95_1_0_20170117174948948.jpg.chip.jpg,90


In [3]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

batch_size = 32
epochs = 50
input_shape = (256, 256, 3)

# 이미지 증강 세팅
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)



# 학습 데이터셋 생성
train_generator = train_datagen.flow_from_dataframe(
    train_data,
    x_col='path',
    y_col='ageRange',
    target_size=input_shape[:2],
    batch_size = batch_size,
    class_mode='categorical'
)

# 검증 데이터셋 생성
test_generator = test_datagen.flow_from_dataframe(
    test_data,
    x_col='path',
    y_col='ageRange',
    target_size=input_shape[:2],
    batch_size = batch_size,
    class_mode='categorical'
)

Found 24000 validated image filenames belonging to 10 classes.
Found 6000 validated image filenames belonging to 10 classes.


<h1>전이 학습</h1>

In [4]:
def get_model(model):
# Load the pretained model
    kwargs =    {'input_shape':input_shape,
                'include_top':False,
                'weights':'imagenet',
                'pooling':'avg'}
    
    pretrained_model = model(**kwargs)
    pretrained_model.trainable = False # 레이어를 동결 시켜서 훈련중 손실을 최소화 한다.
    
    inputs = pretrained_model.input

    x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
    x = tf.keras.layers.Dense(128, activation='relu')(x)

    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [5]:
model = get_model(tf.keras.applications.ResNet152V2)

2023-04-26 19:51:05.448401: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 19:51:05.478172: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 19:51:05.478349: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 19:51:05.478993: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint

# 모델을 학습합니다.
checkpoint_path = "./ResNet152V2_best_model.h5"
checkpoint = ModelCheckpoint(checkpoint_path, 
                             monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')

In [7]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,
    batch_size=batch_size,
    callbacks=[checkpoint]
)

Epoch 1/10


2023-04-26 19:51:37.105933: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8500


750/750 [==============================] - ETA: 0s - loss: 1.5649 - accuracy: 0.3803
Epoch 00001: val_accuracy improved from -inf to 0.44783, saving model to ./ResNet152V2_best_model.h5


/home/uit-na/anaconda3/envs/hello/lib/python3.9/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


750/750 [==============================] - 248s 323ms/step - loss: 1.5649 - accuracy: 0.3803 - val_loss: 1.3858 - val_accuracy: 0.4478
Epoch 2/10
750/750 [==============================] - ETA: 0s - loss: 1.4181 - accuracy: 0.4296
Epoch 00002: val_accuracy did not improve from 0.44783
750/750 [==============================] - 240s 320ms/step - loss: 1.4181 - accuracy: 0.4296 - val_loss: 1.3541 - val_accuracy: 0.4438
Epoch 3/10
750/750 [==============================] - ETA: 0s - loss: 1.3627 - accuracy: 0.4501
Epoch 00003: val_accuracy improved from 0.44783 to 0.46883, saving model to ./ResNet152V2_best_model.h5
750/750 [==============================] - 243s 323ms/step - loss: 1.3627 - accuracy: 0.4501 - val_loss: 1.3068 - val_accuracy: 0.4688
Epoch 4/10
750/750 [==============================] - ETA: 0s - loss: 1.3340 - accuracy: 0.4632
Epoch 00004: val_accuracy improved from 0.46883 to 0.47850, saving model to ./ResNet152V2_best_model.h5
750/750 [==============================] - 2

In [2]:
dataset = pd.read_pickle("./new_dataset.pkl")
dataset

,ageRange,path
18640,0,./UTKFace/26_1_3_20170117153712926.jpg.chip.jpg
8878,0,./UTKFace/35_1_0_20170117092205487.jpg.chip.jpg
7495,0,./UTKFace/30_1_2_20170116192643744.jpg.chip.jpg
3851,0,./UTKFace/31_1_4_20170112235504601.jpg.chip.jpg
3407,0,./UTKFace/36_0_1_20170120221302965.jpg.chip.jpg
...,...,...
18557,3,./UTKFace/85_1_2_20170112204748587.jpg.chip.jpg
14014,3,./UTKFace/85_1_0_20170110182918046.jpg.chip.jpg
2277,3,./UTKFace/80_1_1_20170116153903211.jpg.chip.jpg
18416,3,./UTKFace/85_0_0_20170111210559755.jpg.chip.jpg


In [3]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

batch_size = 32
epochs = 50
input_shape = (256, 256, 3)

# 이미지 증강 세팅
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)



# 학습 데이터셋 생성
train_generator = train_datagen.flow_from_dataframe(
    train_data,
    x_col='path',
    y_col='ageRange',
    target_size=input_shape[:2],
    batch_size = batch_size,
    class_mode='categorical'
)

# 검증 데이터셋 생성
test_generator = test_datagen.flow_from_dataframe(
    test_data,
    x_col='path',
    y_col='ageRange',
    target_size=input_shape[:2],
    batch_size = batch_size,
    class_mode='categorical'
)

Found 9600 validated image filenames belonging to 4 classes.
Found 2400 validated image filenames belonging to 4 classes.


In [4]:
def get_model(model):
# Load the pretained model
    kwargs =    {'input_shape':input_shape,
                'include_top':False,
                'weights':'imagenet',
                'pooling':'avg'}
    
    pretrained_model = model(**kwargs)
    pretrained_model.trainable = False # 레이어를 동결 시켜서 훈련중 손실을 최소화 한다.
    
    inputs = pretrained_model.input

    x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
    x = tf.keras.layers.Dense(128, activation='relu')(x)

    outputs = tf.keras.layers.Dense(4, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [5]:
model = get_model(tf.keras.applications.ResNet152V2)

2023-04-26 23:32:10.566218: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 23:32:10.594313: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 23:32:10.594481: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 23:32:10.594944: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint


# 일반 체크포인트 저장
checkpoint_filepath = './model/ResNet50V2/checkpoint.{epoch:02d}-{val_accuracy:.2f}.h5'

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    save_best_only=False,
    monitor='val_accuracy',
    verbose=1
)

# best 체크포인트 저장
# best_checkpoint_filepath = './model/ResNet50V2/best_checkpoint.{epoch:02d}-{val_loss:.2f}.h5'
# best_checkpoint_callback = ModelCheckpoint(
#     filepath=best_checkpoint_filepath,
#     save_weights_only=True,
#     save_best_only=True,
#     monitor='val_loss',
#     verbose=1
# )

best_checkpoint_filepath = './model/ResNet50V2/best.h5'
best_checkpoint_callback = ModelCheckpoint(
    filepath=best_checkpoint_filepath,
    save_weights_only=False,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

# checkpoint_path = "./model/ResNet50V2/best.h5"
# checkpoint = ModelCheckpoint(checkpoint_path, 
#                              monitor='val_accuracy', 
#                              verbose=1, 
#                              save_best_only=True, 
#                              mode='max')

In [16]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5)

In [17]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=100,
    batch_size=batch_size,
    callbacks=[checkpoint_callback, best_checkpoint_callback, early_stop]
)

Epoch 1/100
300/300 [==============================] - ETA: 0s - loss: 0.5581 - accuracy: 0.7589
Epoch 00001: saving model to ./model/ResNet50V2/checkpoint.01-0.72.h5

Epoch 00001: val_accuracy did not improve from 0.74958
300/300 [==============================] - 98s 327ms/step - loss: 0.5581 - accuracy: 0.7589 - val_loss: 0.6698 - val_accuracy: 0.7171
Epoch 2/100
300/300 [==============================] - ETA: 0s - loss: 0.5640 - accuracy: 0.7530
Epoch 00002: saving model to ./model/ResNet50V2/checkpoint.02-0.73.h5

Epoch 00002: val_accuracy did not improve from 0.74958
300/300 [==============================] - 97s 323ms/step - loss: 0.5640 - accuracy: 0.7530 - val_loss: 0.6569 - val_accuracy: 0.7254
Epoch 3/100
300/300 [==============================] - ETA: 0s - loss: 0.5488 - accuracy: 0.7633
Epoch 00003: saving model to ./model/ResNet50V2/checkpoint.03-0.73.h5

Epoch 00003: val_accuracy did not improve from 0.74958
300/300 [==============================] - 97s 324ms/step - los

In [18]:
from tensorflow.keras.models import load_model

# 이전에 학습한 모델 불러오기
model = load_model('./model/ResNet50V2/best.h5')



In [ ]:
%%javascript
Jupyter.notebook.session.delete();